In [25]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
import os

First, initialize your LLM

In [26]:
model_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(
    model_name=model_name,
    temperature=0,
    max_tokens=2000,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Then, load reviews from tv-reviews.csv

In [27]:
loader = CSVLoader(file_path='./tv-reviews.csv')
docs = loader.load()

Split the documents you loaded into smaller chunks

In [28]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

Now, initialize your embeddings

In [29]:
embeddings = OpenAIEmbeddings()

Initialize your vector db with your embeddings model and populate with your text chunks

In [30]:
db = Chroma.from_documents(split_docs, embeddings)

Query your vector database for 5 most semantically similar chunks

In [31]:
query = """
    Based on the reviews in the context, tell me what people liked about the picture quality.
    Make sure you do not paraphrase the reviews, and only use the information provided in the reviews.
    """

use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))

People liked the sharpness and lifelike quality of the images on the screen, as well as the vibrant and realistic colors that made everything look stunning.
